**This notebook creates the following input for the PySWMM simulations:**
- geometric parameters: area (A), length (L), width (W), pervious fraction (fV), slope (S), previous area (A2), impervious area (A1)
- soil parameters (for Modified Green Ampt infiltration model)
- precipitation scenarios (P)
- anctecedent soil moisture conditions (ASM)

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
import numpy as np;
import pandas as pd;
import os;
import matplotlib.pyplot as plt;
import itertools
import pandas as pd
import numpy as np
import pickle
from itertools import combinations

In [21]:
# A, L, fV: random parameters from normal distribution
# S: 1-4%
# W: A/L

params={'A':np.random.uniform(1,247,100)/100.,               # A : 0 to 2.47105 acre (1 hectare)
        'L':np.random.uniform(1,500,100),                      # L: 0 to 500 ft (per SWMM manual)
        'fV':np.random.uniform(1,100,100)/100.}               # fV: 0 to 100%

#slope from 1 to 4 %
s1=pd.DataFrame(params)
s2=pd.DataFrame(params)
s3=pd.DataFrame(params)
s4=pd.DataFrame(params)

k=0
for s in [s1,s2,s3,s4]:
    k=k+1
    s['S']=k

#  concat all slope dfs, compute W, L:W, A1 and A2
params = pd.concat([s1,s2,s3,s4])
params['W']=(params['A']*43560)/params['L'] #W in ft
params_df['W2']=(params['A']*43560)/params['L'] #W2 = W
params['L:W']=params['L']/params['W'] 
params['A2']=params['fV']*params['A'] # A2 = pervious area
params['A1']=params['A']-params['A2'] # A1 = impervious area

In [3]:
#send to params to pickle
with open('params_2.pickle', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
# soil dictionary (K in in/hr, H_i in in)

soil ={'Soil Texture Class': {0: 'Sand',  1: 'Loamy Sand',  2: 'Sandy Loam',  3: 'Loam',  4: 'Silt Loam',5: 'Sandy Clay Loam',  6: 'Silty Clay Loam',  7: 'Sandy Clay',  8: 'Silty Clay',  9: 'Clay'},
 'K': {0: 4.74,  1: 1.18,  2: 0.43,  3: 0.13,  4: 0.26,  5: 0.06,  6: 0.04,  7: 0.023,  8: 0.02,  9: 0.01},
 'H_i': {0: 1.93,  1: 2.4,  2: 4.33,  3: 3.5,  4: 6.69,  5: 8.66,  6: 10.63,  7: 9.45,  8: 11.42,  9: 12.6},
 'ϕ': {0: 0.437,  1: 0.437,  2: 0.453,  3: 0.463,  4: 0.501,  5: 0.398,  6: 0.471,  7: 0.43,  8: 0.479,  9: 0.475},
 'FC': {0: 0.062,  1: 0.105,  2: 0.19,  3: 0.232,  4: 0.284,  5: 0.244,  6: 0.342,  7: 0.321,  8: 0.371,  9: 0.378},
 'WP': {0: 0.024,  1: 0.047,  2: 0.085,  3: 0.116,  4: 0.135,  5: 0.136,  6: 0.21,  7: 0.221,  8: 0.251,  9: 0.265},
'SAT': {0: 0.437,  1: 0.437,  2: 0.453,  3: 0.463,  4: 0.501,  5: 0.398,  6: 0.471,  7: 0.43,  8: 0.479,  9: 0.475}}

In [60]:
asm=['WP','FC','SAT']

def soil_asm(ASM):
    temp = soil
    temp['IMD']=temp['ϕ'][1]-temp[ASM][1]
    temp['Cond']=ASM
    return temp

soil2={}
for i,a in enumerate(asm):
    soil2[i]=soil_asm(a)
    pd.DataFrame(soil2[i])
WP=pd.DataFrame(soil2[0])
FC=pd.DataFrame(soil2[1])
SAT=pd.DataFrame(soil2[2])
soil_df = pd.concat([WP,FC,SAT])

In [61]:
with open('soil_df.pickle', 'wb') as handle:
    pickle.dump(soil, handle, protocol=pickle.HIGHEST_PROTOCOL)